## High-resolution freeway data from USDOT

-This notebook obtains USDOT road data for San Diego County (FIPS 06073) and 
loads a representations of interstates and state routes covered by PEMS sensors into the AWS RDS database.
- The motivation is to provide visually more appealing representations of traffic conditions and forecasts,
especially for segments with sparse sensor coverage or at high zoom levels
- The data only provides coordinates and road names.
- Although both directions are represented through individual strands, labels do not allow distinguishing them
- Some freeways are made of multiple segments which are not stored in consecutive order.
- This notebook manually assigns road segments (identified by LINEARID) belonging to a given freeway and direction
- Segments are then re-ordered in direction from south to north or west to east, consistent with direction for mile markers.
- Absolute positions are then calculated using the Haversine distance between adjacent points.  As SD county marks the southwestern corner, these should be accurate enough for our purposes.
- The notebook needs to be modified to load Bay area road data.

Data source:
https://data-usdot.opendata.arcgis.com/datasets/census-tiger-line-roads
https://www2.census.gov/geo/tiger/TIGER2019/ROADS/

In [1]:
!ls *.shp

In [51]:
#Download data for SD county, FIPS code 06073
#!curl -O https://www2.census.gov/geo/tiger/TIGER2019/ROADS/tl_2019_06073_roads.zip
#!unzip tl_2019_06073_roads.zip

In [52]:
fips='06073'
myshp = open("tl_2019_%s_roads.shp" % fips, "rb")
mydbf = open("tl_2019_%s_roads.dbf" % fips, "rb")

In [53]:
import shapefile
r = shapefile.Reader(shp=myshp, dbf=mydbf)

In [54]:
r.shapeRecords

<bound method Reader.shapeRecords of <shapefile.Reader object at 0x7fde4623da00>>

In [55]:
#for a in r.iterRecords():
#    print(a)
sh=[a for a in r.iterShapeRecords()]
#print(r)

In [56]:
type(sh[0])

shapefile._ShapeRecord

In [57]:
import plotly.express as px
import plotly
plotly.io.templates.default='plotly_dark'

In [58]:
#sh[2].__geo_interface__
print(sh[201].record)
print(sh[201].shape.__geo_interface__)

['1102213379838', 'Old Stonefield Chase', 'M', 'S1400']
{'type': 'LineString', 'coordinates': ((-117.140201, 33.016787), (-117.140075, 33.016684999999995), (-117.14001, 33.01664399999999), (-117.13991500000002, 33.01660199999999), (-117.139813, 33.016573), (-117.139706, 33.016559), (-117.138849, 33.016563999999995), (-117.137993, 33.01657), (-117.13776700000001, 33.016577999999996), (-117.137668, 33.01660799999999), (-117.137548, 33.01665899999999), (-117.137419, 33.016719), (-117.13732900000001, 33.01674899999999), (-117.137214, 33.016761999999986), (-117.137029, 33.016761999999986), (-117.136707, 33.016774999999996), (-117.136604, 33.01677099999999), (-117.13648, 33.01677099999999), (-117.136377, 33.01676599999999), (-117.136231, 33.016757999999996), (-117.136128, 33.016761999999986), (-117.136047, 33.016778999999985), (-117.135965, 33.01682699999999), (-117.135935, 33.01687799999999), (-117.135927, 33.01692899999999), (-117.135927, 33.017352999999986))}


In [59]:
sh[3].shape.__geo_interface__

{'type': 'LineString',
 'coordinates': ((-117.259592, 33.186980999999996),
  (-117.259481, 33.18698799999999),
  (-117.259419, 33.186982),
  (-117.25935400000002, 33.186980999999996),
  (-117.259293, 33.18697999999999),
  (-117.259225, 33.18697699999999),
  (-117.259166, 33.18697699999999),
  (-117.259105, 33.186976),
  (-117.259049, 33.18696599999999),
  (-117.25897900000001, 33.18696599999999),
  (-117.25892000000002, 33.186963999999996),
  (-117.258833, 33.186962),
  (-117.258775, 33.18696299999999),
  (-117.258719, 33.186963999999996),
  (-117.258659, 33.186970999999986),
  (-117.25857600000002, 33.186973),
  (-117.25852200000001, 33.18696599999999),
  (-117.25846, 33.186963999999996))}

In [60]:
sh[2].record

['1103663797349', 'McGavran Dr Exn', 'M', 'S1400']

In [61]:
#import pandas as pd
#adf=pd.DataFrame(columns=["lat","lon","name"])

#p=0

import pandas as pd

lons=[]
lats=[]
roads=[]
lineids=[]
inf3=[]
inf4=[]

for ash in sh:
    #lineid=ash.__geo_interface__["properties"]["LINEARID"]
    #roadname=ash.__geo_interface__["properties"]["FULLNAME"]
    lineid=ash.record[0]
    roadname=str(ash.record[1])
    tmp3=str(ash.record[2])
    tmp4=str(ash.record[3])
    for pt in ash.shape.__geo_interface__["coordinates"]:
        lons.append(pt[0])
        lats.append(pt[1])
        roads.append(roadname)
        lineids.append(lineid)
        inf3.append(tmp3)
        inf4.append(tmp4)
    
adf=pd.DataFrame({"lat": lats, "lon": lons, "name": roads, "LINEARID": lineids, "inf3": inf3, "inf4": inf4})
adf.head()

,lat,lon,name,LINEARID,inf3,inf4
0,32.770776,-117.162913,Friars Rd Rmp,110686150541,M,S1400
1,32.770853,-117.162952,Friars Rd Rmp,110686150541,M,S1400
2,32.771084,-117.163070,Friars Rd Rmp,110686150541,M,S1400
3,32.771162,-117.163110,Friars Rd Rmp,110686150541,M,S1400
4,32.771250,-117.163155,Friars Rd Rmp,110686150541,M,S1400


In [62]:
adf.tail()
adf.loc[1309590,"name"]

"b'                                                                                                    '"

In [63]:
adf[adf["name"].str.contains('805')]

,lat,lon,name,LINEARID,inf3,inf4
162292,32.545708,-117.034246,I- 805,1104483424018,I,S1100
162293,32.546205,-117.034945,I- 805,1104483424018,I,S1100
162294,32.548720,-117.038335,I- 805,1104483424018,I,S1100
162295,32.549419,-117.039137,I- 805,1104483424018,I,S1100
162296,32.549747,-117.039472,I- 805,1104483424018,I,S1100
...,...,...,...,...,...,...
1249292,32.904902,-117.224518,I- 805,1108311288468,I,S1100
1249293,32.905695,-117.225090,I- 805,1108311288468,I,S1100
1249294,32.906043,-117.225306,I- 805,1108311288468,I,S1100
1249295,32.906978,-117.225833,I- 805,1108311288468,I,S1100


It looks like interstates are named `I- number`, e.g. I- 805, I- 5, with a space between the hyphen and number

In [64]:
adf[adf["name"].str.contains("I- 15")]

,lat,lon,name,LINEARID,inf3,inf4
296890,32.852204,-117.115323,I- 15,1105281362743,I,S1100
296891,32.852886,-117.115203,I- 15,1105281362743,I,S1100
296892,32.853752,-117.114974,I- 15,1105281362743,I,S1100
296893,32.854646,-117.114705,I- 15,1105281362743,I,S1100
296894,32.854851,-117.114635,I- 15,1105281362743,I,S1100
...,...,...,...,...,...,...
1256711,32.849227,-117.116325,I- 15,1105056901671,I,S1100
1256712,32.850703,-117.115829,I- 15,1105056901671,I,S1100
1256713,32.851872,-117.115434,I- 15,1105056901671,I,S1100
1256714,32.852129,-117.115348,I- 15,1105056901671,I,S1100


In [65]:
def plot_fwy(fwlabel):
    #sel=adf[adf["name"].str.contains(fwlabel)]
    sel=adf[adf["name"] == fwlabel]
    fig=px.line_mapbox(sel, lat="lat", lon="lon", line_group="LINEARID",
                   mapbox_style="carto-darkmatter", color="LINEARID")
    
    print(sel["LINEARID"].unique())
    print(sel["name"].unique())
    
    return fig 
            
plot_fwy("I- 5")

['1105078975119' '1104471909281' '1105078810436' '1104474157403'
 '1105078975118']
['I- 5']


In [66]:
adf[adf["inf3"] == "I"]["name"].unique()

array(['I- 5 Local Byp', 'I- 805', 'I- 8', 'I- 15', 'I- 5'], dtype=object)

## Tasks:  

- manually / automatically identify which LINEARIDs belong to which freeway and direction in the database
- sometimes (such as in the I-15) the LINEARID changes for the same fwy and direction
- come up with a list of coordinates for each fwy-direction combination in the database
- interpolate abs_pm into those segments

In [67]:
import pickle
fwdir=pickle.load(open("SD_fwys.pkl","rb"))
print(fwdir)

['94W', '125S', '78E', '163S', '8W', '8E', '125N', '94E', '805S', '78W', '15N', '5S', '15S', '5N', '805N', '163N', '54E', '54W', '56E', '56W', '52E', '52W', '67S', '67N', '905W', '905E', '76W', '76E', '11E', '11W']


In [68]:
liddf=pd.DataFrame(columns=["FWY","DIR","IDS"])
for n,fwd in enumerate(fwdir):
    liddf.loc[n,"DIR"] = fwd[-1] 
    liddf.loc[n,"FWY"] = fwd.rstrip(liddf.loc[n,"DIR"])
    liddf.loc[n,"IDS"] = []
liddf.sort_values(by="FWY", inplace=True)
#liddf.reset_index(inplace=True, drop=True)
liddf.set_index(["FWY","DIR"], inplace=True)

In [69]:
liddf.head(n=1)

,,IDS
FWY,DIR,
11,W,[]


In [70]:
liddf.loc[("11","W"),:]

IDS    []
Name: (11, W), dtype: object

In [71]:
#manually assigning ids(s) to fwy and direction
liddf.loc[('11',"W"), "IDS" ] = ['1106048350799']
liddf.loc[('11',"E"), "IDS"] = ['1106048206346']
liddf.loc[('125',"S"), "IDS"] = ['1108296282831', '1108296281611', '1108311437048']
liddf.loc[('125',"N"), "IDS"] = ['11013164234852', '1108296281647', '1108296282824']
liddf.loc[('15',"S"), "IDS"] = ['1105281384965']
liddf.loc[('15',"N"), "IDS"] = ['1105056901671', '1105057650128']
liddf.loc[('163',"S"), "IDS"] = ['1108296282856', '1106072574446']
liddf.loc[('163',"N"), "IDS"] = ['1104257635336', '1108296282915']
liddf.loc[('5',"S"), "IDS"] = ['1104471909281']
liddf.loc[('5',"N"), "IDS"] = ['1104474157403']
liddf.loc[('52',"W"), "IDS"] = ['1108311436977']
liddf.loc[('52',"E"), "IDS"] = ['1108296281525']
liddf.loc[('54',"W"), "IDS"] = ['1108296281839'] #'1104258657861', '1104258308692' - no sensors in these segments
liddf.loc[('54',"E"), "IDS"] = ['1108312284493'] #, '1104258304425', '1104258657863', '1104259380421']
liddf.loc[('56',"W"), "IDS"] = ['1108312012637']
liddf.loc[('56',"E"), "IDS"] = ['1108312012633']
liddf.loc[('67',"S"), "IDS"] = ['1108296282761']
liddf.loc[('67',"N"), "IDS"] = ['1108296282741']
liddf.loc[('76',"W"), "IDS"] = ['1104257714520', '1108297200688', '11012814385028'] #no direction separation
liddf.loc[('76',"E"), "IDS"] = ['1104257714521', '1108297200688', '11012814385028']
liddf.loc[('78',"W"), "IDS"] = ['1109090122891']
liddf.loc[('78',"E"), "IDS"] = ['1108311364685']
liddf.loc[('8',"W"), "IDS"] = ['1104470395751']
liddf.loc[('8',"E"), "IDS"] = ['1104259213007']
liddf.loc[('805',"S"), "IDS"] = ['1108311288468']
liddf.loc[('805',"N"), "IDS"] = ['1104483424018']
liddf.loc[('905',"W"), "IDS"] = ['1108311437108']
liddf.loc[('905',"E"), "IDS"] = ['1108311437105']
liddf.loc[('94',"W"), "IDS"] = ['1108296282470']
liddf.loc[('94',"E"), "IDS"] = ['1108296282496']

In [72]:
fwys=liddf.index.get_level_values(0).unique()

In [73]:
#plot_fwy("I- 8")
plot_fwy("State Rte 76")

['11012814385028' '1104257714520' '1108297200688' '1104257714521']
['State Rte 76']


In [74]:
# attempted to write a function which does the assignment automatically
# ended up doing it manually, as there are too many inconsistencies
# (multiple segments per direction, local bypasses, separate ids for carpool lanes,
#  or single strand roads when directions are not separated)
def assign_ids(fwy, dirs):
    if fwy in ['5', '15', '805', '8']:
        mask=adf["name"].str.contains("I- %s" % fwy)
    else:
        mask=adf["name"].str.contains("State Rte %s" % fwy)
    sel=adf[mask]
    lineids=sel["LINEARID"].unique()
    
    #pick the two longest strands
    
    
    if dirs[0] == "W" or dirs[1] == "W":
        pass
        
    return lineids

assign_ids("11", ["N","S"])

array(['1106048350799', '1106048206346'], dtype=object)

In [75]:
liddf

IDS
FWY DIR                                                
11  W                                   [1106048350799]
    E                                   [1106048206346]
125 S     [1108296282831, 1108296281611, 1108311437048]
    N    [11013164234852, 1108296281647, 1108296282824]
15  S                                   [1105281384965]
    N                    [1105056901671, 1105057650128]
163 N                    [1104257635336, 1108296282915]
    S                    [1108296282856, 1106072574446]
5   N                                   [1104474157403]
    S                                   [1104471909281]
52  E                                   [1108296281525]
    W                                   [1108311436977]
54  E                                   [1108312284493]
    W                                   [1108296281839]
56  E                                   [1108312012633]
    W                                   [1108312012637]
67  S                                   [1108296282761]
    N                                   [1108296282741]
76  W    [1104257714520, 1108297200688, 11012814385028]
    E    [1104257714521, 1108297200688, 11012814385028]
78  E                                   [1108311364685]
    W                                   [1109090122891]
8   E                                   [1104259213007]
    W                                   [1104470395751]
805 N                                   [1104483424018]
    S                                   [1108311288468]
905 E                                   [1108311437105]
    W                                   [1108311437108]
94  E                                   [1108296282496]
    W                                   [1108296282470]

In [76]:
liddf.to_pickle("TIGER_PEMS_SD_IDS.pkl")

In [77]:
rte=liddf.index[0]
liddf.loc[rte,:]

for lineid in liddf.loc[rte,"IDS"]:
    print(adf[adf["LINEARID"] == lineid]["lat"].values)

[32.56406  32.564004 32.564019 32.56406  32.564064 32.564093 32.564149
 32.564184 32.564249]


In [78]:
list([1,2]) + list([3,4])

[1, 2, 3, 4]

In [79]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [80]:
#haversine_distances?

In [81]:
earth_radius=6371.
miles_per_km=1.609344
def get_relative_pm(indf):
    #relative position
    indf["rel_pm"] = 0.
    for n in indf.index[1:]:
        cp=[radians(indf.loc[n,"latitude"]), radians(indf.loc[n,"longitude"])]
        pp=[radians(indf.loc[n-1,"latitude"]), radians(indf.loc[n-1,"longitude"])]
        result=haversine_distances([cp, pp]) * earth_radius / miles_per_km
        indf.loc[n,"rel_pm"] = indf.loc[n-1,"rel_pm"] + result[1][0]
    return indf

In [82]:
def add_rte_coordinates():
    fwy_hr={}
    for rte in liddf.index:
        fwy=rte[0]
        direc=rte[1]
        fwstr=fwy+direc
        
        lats=[]
        lons=[]
        #lats=pd.concat([adf[adf["LINEARID"] == lineid]["lat"]] for lineid in liddf.loc[rte,"IDS"])
        for lineid in liddf.loc[rte,"IDS"]:
            lats = lats + list(adf[adf["LINEARID"] == lineid]["lat"].values)
            lons = lons + list(adf[adf["LINEARID"] == lineid]["lon"].values)
            
        tmpdf = pd.DataFrame({"latitude": lats, "longitude": lons})
        fwy_hr[fwstr] = get_relative_pm(tmpdf)
        #    mask=adf["LINEARID"] == lineid
        #    lons=
        
    return fwy_hr

fwy_hr=add_rte_coordinates()

Need a way to organize different segments into coherent route
Options:
1. Treat all points as independent (ignore LINEARID) and solve as TSP
2. Start with southernmost point for SB/NB fwy and subsequently find closest neighbor
3. Figure out which segment is most south / west.  Take as is if first point is furthest south or west, otherwise flipud.
   Then take second northernmost segment.  Flip if needed.  Continue.
   
Using option number 3 here

Note: mile marker numbers always get larger as you travel east or north (USDOT).  Therefore, absolute marker numbers computed for SD county should already be quite close to PEMS value, as SD is in the southwestern corner.

In [83]:
import numpy as np

#same as add_rte_coordinates, but ordering segments
#Mile marker numbers always get larger as you travel east or north (usdot)
def add_rte_coordinates2():
    fwy_hr={}
    for rte in liddf.index:
        fwy=rte[0]
        direc=rte[1]
        fwstr=fwy+direc
        
        linearid=[lineind for lineind in liddf.loc[rte,"IDS"]]
        latmean=[adf[adf["LINEARID"] == lineind]["lat"].values.mean() for lineind in liddf.loc[rte,"IDS"]]
        lonmean=[adf[adf["LINEARID"] == lineind]["lon"].values.mean() for lineind in liddf.loc[rte,"IDS"]]
        llmean=pd.DataFrame({"lat": latmean, "lon": lonmean}, index=linearid)
        
        """
        if direc == "S":
            llmean=llmean.sort_values(by="lat", ascending=False)
        elif direc == "N":
            llmean=llmean.sort_values(by="lat", ascending=True)
        elif direc == "E":
            llmean=llmean.sort_values(by="lon", ascending=False)
        elif direc == "W":
            llmean=llmean.sort_values(by="lon", ascending=True)
        """        
        if direc == "S" or direc == "N":
            llmean=llmean.sort_values(by="lat", ascending=True)
        elif direc == "E" or direc == "W":
            llmean=llmean.sort_values(by="lon", ascending=True)
        else:
            print("Unknown direction.")
        
        lats=[]
        lons=[]
        
        for lineid in llmean.index:
            newlat=adf[adf["LINEARID"] == lineid]["lat"].values
            newlon=adf[adf["LINEARID"] == lineid]["lon"].values
            
            if (direc == "S" and newlat[0] > newlat[-1]) or \
               (direc == "N" and newlat[0] > newlat[-1]) or \
               (direc == "W" and newlon[0] > newlon[-1]) or \
               (direc == "E" and newlon[0] > newlon[-1]):
                    print("flipping direction on segment %s" % lineid)
                    newlat=np.flipud(newlat)
                    newlon=np.flipud(newlon)
                
            lats = lats + list(newlat)
            lons = lons + list(newlon)
            
        tmpdf = pd.DataFrame({"latitude": lats, "longitude": lons})
        fwy_hr[fwstr] = get_relative_pm(tmpdf)
        
    return fwy_hr #llmean, newlat

fwy_hr = add_rte_coordinates2()

flipping direction on segment 1106048350799
flipping direction on segment 1108296281611
flipping direction on segment 1108296282856
flipping direction on segment 1104474157403
flipping direction on segment 1104471909281
flipping direction on segment 1108312012633
flipping direction on segment 1108296282761
flipping direction on segment 1104257714520
flipping direction on segment 1109090122891
flipping direction on segment 1108311437108


In [84]:
fwy_hr["5S"]

,latitude,longitude,rel_pm
0,32.542399,-117.029881,0.000000
1,32.542462,-117.029976,0.007040
2,32.542532,-117.030083,0.014929
3,32.542627,-117.030226,0.025534
4,32.542682,-117.030309,0.031683
...,...,...,...
1591,33.391407,-117.590288,71.895145
1592,33.392224,-117.590932,71.962722
1593,33.392980,-117.591529,72.025288
1594,33.395277,-117.593121,72.208651


In [85]:
#adf[adf["LINEARID"] == '1108296282470']["lat"].tolist()
import pickle
with open("TIGER_PEMS_SD_ll_rel_pm.pkl", "wb") as fid:
    pickle.dump(fwy_hr, fid)

In [86]:
!ls -l *.pkl

-rw-rw-r-- 1 daniel daniel    122 May 27 11:27 SC_fwys.pkl
-rw-rw-r-- 1 daniel daniel    200 May 24 21:41 SD_fwys.pkl
-rw-rw-r-- 1 daniel daniel   1321 May 27 11:53 TIGER_PEMS_SC_IDS.pkl
-rw-rw-r-- 1 daniel daniel 194572 May 27 12:46 TIGER_PEMS_SC_ll_rel_pm.pkl
-rw-rw-r-- 1 daniel daniel   1765 May 27 12:50 TIGER_PEMS_SD_IDS.pkl
-rw-rw-r-- 1 daniel daniel 521844 May 27 12:51 TIGER_PEMS_SD_ll_rel_pm.pkl


In [87]:
host="capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
username="group3"
db="db1"
#pw=getpass.getpass("Enter database password")
pw=open("/home/daniel/Desktop/.awsdb","r").read().rstrip()

In [88]:
import sqlalchemy as sal
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (username, pw, host, db))

In [89]:
query="""
select p.latitude as lat, p.longitude as lon, p.abs_pm
from pemslocs p 
where p.fwy=76 and p.direc='W' and p.stype='ML' and p.county = 73
order by abs_pm
"""
sensors=pd.read_sql(query,  engine)

In [90]:
import plotly.graph_objects as go
sensors["color"] = 'PEMS'
fig=px.scatter_mapbox(sensors, lat="lat", lon="lon",
                 mapbox_style="carto-darkmatter",
                 hover_data=["abs_pm"], color="color",
                 color_discrete_map={"PEMS": "red"})

sel=fwy_hr["76E"]
sel["color"] = "USDOT"
fig2=px.line_mapbox(sel, lat="latitude", lon="longitude",
                      hover_data=["rel_pm"],
                      color="color",
                      color_discrete_map={"USDOT": "green"})

fig.add_trace(fig2.data[0])
#fig

In [91]:
liddf.loc[('125','S'),:]

IDS    [1108296282831, 1108296281611, 1108311437048]
Name: (125, S), dtype: object

In [92]:
pd.read_sql("select * from pemslocs limit 1", engine)

,sid,fwy,direc,district,county,city,state_pm,abs_pm,latitude,longitude,length,stype,lanes,name
0,308511,50,E,3,17,None,31.627,60.162,38.761062,-120.569835,5.0,ML,2,Sly Park Rd


In [93]:
adf.query("LINEARID == '1108296281611'")

,lat,lon,name,LINEARID,inf3,inf4
787825,32.747805,-117.018261,State Rte 125,1108296281611,S,S1100
787826,32.747401,-117.018427,State Rte 125,1108296281611,S,S1100
787827,32.747031,-117.018503,State Rte 125,1108296281611,S,S1100
787828,32.746460,-117.018565,State Rte 125,1108296281611,S,S1100
787829,32.746223,-117.018586,State Rte 125,1108296281611,S,S1100
...,...,...,...,...,...,...
787909,32.700404,-117.011598,State Rte 125,1108296281611,S,S1100
787910,32.699786,-117.011823,State Rte 125,1108296281611,S,S1100
787911,32.698968,-117.012150,State Rte 125,1108296281611,S,S1100
787912,32.697965,-117.012618,State Rte 125,1108296281611,S,S1100


In [94]:
#merge into single dataframe

fwydf=pd.DataFrame(columns=["fwdir", "latitude", "longitude", "abs_pm"])

for fwk in fwy_hr.keys():
    ndf=fwy_hr[fwk].copy()
    ndf["fwdir"] = fwk
    newcol=list(ndf.columns)
    newcol[2] = "abs_pm"
    ndf.columns = newcol
    fwydf=fwydf.append(ndf[fwydf.columns])
fwydf.reset_index(drop=True, inplace=True)

In [95]:
for col in ["latitude","longitude","abs_pm"]:
    fwydf[col] = fwydf[col].astype(float)

In [100]:
with engine.connect() as con:
    con.execute("drop table usdot")

In [101]:
create_cmd="""
create table usdot (
    usdid serial primary key, 
    fwdir varchar not null,  
    latitude float not null,  
    longitude float not null,  
    abs_pm float);
"""

with engine.connect() as con:
    con.execute(create_cmd)

In [102]:
fwydf.to_sql("usdot", engine, if_exists = 'append', method='multi', index=False, index_label="usdid")

In [103]:
pd.read_sql("select * from usdot", engine, index_col="usdid").tail()

,fwdir,latitude,longitude,abs_pm
usdid,,,,
21386,94W,32.745322,-116.962812,11.581083
21387,94W,32.745417,-116.962432,11.604121
21388,94W,32.745441,-116.962344,11.609497
21389,94W,32.745517,-116.962081,11.625658
21390,94W,32.745543,-116.961994,11.631023
